# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [20]:
pwd

'C:\\Users\\sravi\\Desktop\\Data Analytics BootCamp\\GIT_Files\\HW_python-api-challenge\\python-api-challenge\\VacationPy'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [21]:
city_weather_data = pd.read_csv("../WeatherPy/output_data/cities.csv")
city_weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Erdemli,36.6050,34.3084,75.78,81,0,1.86,TR,1655755261
1,Turan,16.0678,108.2208,84.18,70,20,0.00,VN,1655754997
2,Helden,51.1150,7.9519,60.39,75,0,3.96,DE,1655755261
3,Atuona,-9.8000,-139.0333,78.12,81,8,25.79,PF,1655755008
4,Pontes e Lacerda,-15.2261,-59.3353,90.14,34,16,4.36,BR,1655755028
...,...,...,...,...,...,...,...,...,...
545,St. George's,12.0564,-61.7485,85.68,62,20,19.57,GD,1655755400
546,Boddam,57.4667,-1.7833,62.98,86,100,8.34,GB,1655755400
547,Davila,18.4726,120.5763,82.38,77,100,4.34,PH,1655755400
548,Aden,12.7794,45.0367,89.37,72,87,12.41,YE,1655755205


In [22]:
print(f"Before removing the NA values, there are {len(city_weather_data)} cities in the dataset.")
city_weather_clean = city_weather_data.dropna()
print(f"After removing the NA values, there are {len(city_weather_clean)} cities in the dataset.")

Before removing the NA values, there are 550 cities in the dataset.
After removing the NA values, there are 546 cities in the dataset.


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [41]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in location's Coordinates
coordinates = city_weather_clean[["Lat", "Lng"]]

# Store location's Humidity in humidity
humidity = city_weather_clean["Humidity"]

In [42]:
fig_heat_layer = gmaps.figure()

heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, dissipating=False, max_intensity=100, point_radius = 2)
fig_heat_layer.add_layer(heat_layer)

fig_heat_layer

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [43]:
# List of Ideal weather conditions: 
# Temp: 70 -85
# Wind Speed: less than 8 mph
# Cloudiness: less than 30%

ideal_city_weather_df = city_weather_clean.loc[(city_weather_clean["Wind Speed"] <= 5) & (city_weather_clean["Cloudiness"] <= 10) & \
                                   (city_weather_clean["Max Temp"] >= 70) & (city_weather_clean["Max Temp"] <= 80)]

ideal_city_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Erdemli,36.6050,34.3084,75.78,81,0,1.86,TR,1655755261
129,Asău,46.4333,26.4000,70.72,76,0,4.97,RO,1655755258
201,Grafton,42.2070,-71.6856,79.52,33,4,4.00,US,1655755307
230,Marsá Maţrūḩ,31.3525,27.2453,74.75,74,0,3.71,EG,1655755316
266,Tunceli,39.1083,39.5471,73.90,41,0,4.47,TR,1655755325
285,Horadiz,39.4488,47.3353,74.66,55,0,4.90,AZ,1655755329
399,Odienné,9.5100,-7.5692,74.12,88,7,3.69,CI,1655755362
426,Aksu,41.1231,80.2644,70.47,37,6,3.49,CN,1655755369
507,Tiznit Province,29.5833,-9.5000,70.20,66,1,4.50,MA,1655755390
527,Rodolfo Sánchez Taboada,31.7167,-116.5667,74.10,74,1,4.00,MX,1655755395


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [64]:
# Create a hotel_df
hotel_df = ideal_city_weather_df.copy()
#hotel_df
hotel_df['Hotel Name']=""

In [65]:
#base url for places
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#setting up parameters for the search.. according to documentation, 'lodging' is the closest parameter for hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

In [66]:
for index, row in hotel_df.iterrows():
    # read lat and lng values from dataframe
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    #change lat and lng as per city while remaining params stay intact
    params["location"] = f"{lat},{lng}"
    # run a request using our params dictionary and convert to json
    response = requests.get(base_url, params=params).json()
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No Hotel Found"

In [67]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Erdemli,36.6050,34.3084,75.78,81,0,1.86,TR,1655755261,Sun Rice Otel
129,Asău,46.4333,26.4000,70.72,76,0,4.97,RO,1655755258,Casa Mika Ciobanus
201,Grafton,42.2070,-71.6856,79.52,33,4,4.00,US,1655755307,The Grafton Inn
230,Marsá Maţrūḩ,31.3525,27.2453,74.75,74,0,3.71,EG,1655755316,Beau Site
266,Tunceli,39.1083,39.5471,73.90,41,0,4.47,TR,1655755325,Grand Hotel Şaroğlu
285,Horadiz,39.4488,47.3353,74.66,55,0,4.90,AZ,1655755329,Horadiz Olympic Center
399,Odienné,9.5100,-7.5692,74.12,88,7,3.69,CI,1655755362,Oustaz Vame DIAKITE
426,Aksu,41.1231,80.2644,70.47,37,6,3.49,CN,1655755369,Pudong Holiday Hotel
507,Tiznit Province,29.5833,-9.5000,70.20,66,1,4.50,MA,1655755390,No Hotel Found
527,Rodolfo Sánchez Taboada,31.7167,-116.5667,74.10,74,1,4.00,MX,1655755395,Casa De Thompson


In [68]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
coordinates = hotel_df[["Lat", "Lng"]]

In [69]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(coordinates, info_box_content=hotel_info)
fig_heat_layer.add_layer(markers)
fig_heat_layer

# Display figure


Figure(layout=FigureLayout(height='420px'))